# Qode - Headhunter Assistant AI


## Use case

Instead of the traditional upload and apply process for candidates. We can create Qode - Headhunter Assistant AI, that can:

- Extract and validate candidate’s information, working experience.

- Summary candidate highlights.

- Ask screening questions such as expected salary, reasons for leaving, motivation for opportunities

- Suggest matched jobs for them (later phase)

## Overview

The chat model interface is based around messages rather than raw text. Several components are important to consider for chat:

* `chat model`: See [here](/docs/integrations/chat) for a list of chat model integrations and [here](/docs/modules/model_io/models/chat) for documentation on the chat model interface in LangChain. You can use `LLMs` (see [here](/docs/modules/model_io/models/llms)) for chatbots as well, but chat models have a more conversational tone and natively support a message interface.
* `prompt template`: Prompt templates make it easy to assemble prompts that combine default messages, user input, chat history, and (optionally) additional retrieved context.
* `memory`: [See here](/docs/modules/memory/) for in-depth documentation on memory types
* `retriever` (optional): [See here](/docs/modules/data_connection/retrievers) for in-depth documentation on retrieval systems. These are useful if you want to build a chatbot with domain-specific knowledge.

## Memory 

As we mentioned above, the core component of chatbots is the memory system. One of the simplest and most commonly used forms of memory is `ConversationBufferMemory`:
* This memory allows for storing of messages in a `buffer`
* When called in a chain, it returns all of the messages it has stored

LangChain comes with many other types of memory, too. [See here](/docs/modules/memory/) for in-depth documentation on memory types.


## Conversation 

We can unpack what goes under the hood with `ConversationChain`. 

We can specify our memory, `ConversationSummaryMemory` and we can specify the prompt. 

## Set up environment

In [ ]:
!pip install langchain openai 

In [1]:
from dotenv import load_dotenv
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI

# Set env var OPENAI_API_KEY or load from a .env file:
load_dotenv()

True

In [8]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [15]:
def print_memory(memory):
    for message in memory.buffer:
        if message.__class__.__name__ == 'AIMessage':
            print(f"\033[92m{message.__class__.__name__}: \033[0m" + message.content)
        else:
            print(f"\033[91m{message.__class__.__name__}: \033[0m" + message.content)
        print("-----")  

## Set up data

In [2]:
resume = """
Email: 		nguyenphuquy2303@gmail.com
Phone: 		+84 979 757 305
LinkedIn:		https://www.linkedin.com/in/nguyenphuquy/
NGUYEN PHU QUY
PROFESSIONAL PROFILE
Results-driven senior data analyst with 5+ years of experience leveraging data-driven insights to optimize strategies and drive operational efficiency in SaaS, e-commerce industry. Skilled in python, machine learning, and data analytics. Proven track record in leading cross-functional teams, delivering measurable outcomes, and effectively communicating solutions.
EXPERIENCE
Kilo MDC — Senior Data Analyst 					          November 2022 – Present
● Initiate a Product Category Matching project aimed at grouping related products into variant and parent groups by using SVM machine learning algorithm. The primary objective is to enhance the user experience on our platform by optimizing product search functionality and sorting results based on the best price offers from sellers.
● Leverage historical order data to identify optimal product combinations that maximize gross profit per order. Factors considered include product price, category take rate, unit size, and logistic cost.
● Utilize GPS data from drivers and retailers' addresses to identify and detect abnormal activities from third-party logistic vendors. Uncover and address cases of cheating activities, which have resulted in financial exploitation of the company amounting to $50k, involving three identified vendors.
Katalon — Data Analyst						        November 2021 – November 2022
● Work with the management team on a pricing analysis project to simulate and evaluate an optimal price for the SaaS seat-based model of Katalon Studio, which helps increase 30% of revenue and streamline sales processes with the new business model.
● Collaborate with the Marketing and Product team on a product qualified leads (PQL) project to build a prediction model on users’ usage patterns, and define a list of likely-to-convert customers. The project contributes to an increase in conversion rate from 1.1% to 1.8% compared to the previous year and reduces the sales cycle from 90 days for regular customers to 45 days for PQL customers.
● Initiate Customer Success Health Score project to help customer managers to identify potential churn risks or expansion opportunities by centralizing and leveraging data on product adoption, product experience, customer engagement, and customer sentiment on Salesforce.
● Create star schema data modeling and data mart for building real-time Power BI dashboards to monitor team’s performance and take proactive actions with insightful trends.
KEY ACHIEVEMENT: Co-lead with a data engineer on building a single source of truth data warehouse of the company with Redshift, in charge of building ETL script, data logic, data models, and documents for business users to self-serve the data, and training users on using BI tools. The team’s workload on repeated data requests is reduced by 70%.
Intel Vietnam — Business Data Analyst 			 	January 2021 – November 2021
● Visualize and automate Customs-Accounting assets reconciliation process with Power BI to monitor customs compliance and detect business risks, which helps to reduce reconciliation cycle from 120 days to 7 days.
● Build assets tracking and aging indicator dashboard to optimize 30% of warehouse space utilization and improve operational efficiency.
● Lead assets management business process improvement project, drive collaboration across Accounting, Engineering, Logistics, IT departments to minimize human-dependent errors from over 10 cases to 2 cases per quarter, and align Intel’s business with the Government regulation.
Intel Vietnam — Senior Process Engineer 				  January 2020 – January 2021
● Build automated Python script for collecting and cleaning data to drive and achieve 99% machine availability. 
● Investigate the root cause of quality issues through commonality analysis and model-based problem solving. 
● Create dashboards to monitor product’s yield and auto trigger problematic trends.
Ascenx — Field Service Engineer 				        December 2015 – December 2019 
ANTS — Account Executive 						 August 2014 - November 2015
EDUCATION & SKILLS
Master of Data Science, Ho Chi Minh City University of Technology
2022 - Expected to graduate in 2024 
Chemical Engineering, Ho Chi Minh City University of Technology
Graduated in October 2014
Internship, Arava International Training Center, Israel 
September 2013 - June 2014
Skills: Data analysis, Data modeling, SQL, Python, Pandas, Airflow
Tools: Power BI, PostgreSQL, Redshift, Dbt, Metabase, Dbeaver, Google Analytics, Hubspot.
"""

## Set up Chatbot

In [62]:

def make_chatbot():
    # LLM
    llm = ChatOpenAI()

    # Prompt 
    prompt = ChatPromptTemplate(
        messages=[
            SystemMessagePromptTemplate.from_template(
    """
    I want to act as a headhunter assistant to validate and ask screening question to candidate.
    At the beginning, you will read the candidate resume. 

    Resume: {}

    Then you will extract and validate those information with user. 

    Step 1: Extract candidate name, phone, email, city, linkedin link, github link, working experiences, Ctop 5 technical skills in below format.

    - Name: <name>
    - Phone: <phone>
    - Email: <email>
    - City: <city>
    - LinkedIn: <LinkedIn link>
    - Github: <Github link>
    - Top 5 skills: <list of skills>
    - Working experience:
        - <company> - <title> - <working date>

    You should ask them if they are correct or not, if not tell them to input correct information.

    Step 2: If candidate update their information, you will show the information again for validation. 
    If candidate doesn't update anything, just move to step 3.

    Step 3: You must ask candidate about gross annual current salary and gross annual expected salary for new job.

    Step 4: You must ask candidate about notice period for current job.

    Step 5: You must ask candidate about reason for leaving current or previous job.

    Step 6: You must ask candidate about motivation for applying for this job.

    Step 7: You must show all the summarise of the candidate for final validation.

    Step 8: Say thank you and encouragement to candidate, and inform them you have applied for the job.

    You must ask those above steps one by one.
    I want you use the tone funny, encouragement and supportive.
    """.format(resume)
            ),
            # The `variable_name` here is what must align with memory
            MessagesPlaceholder(variable_name="chat_history"),
            HumanMessagePromptTemplate.from_template("{answer}")
        ]
    )

    # Notice that we `return_messages=True` to fit into the MessagesPlaceholder
    # Notice that `"chat_history"` aligns with the MessagesPlaceholder name
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

    memory.chat_memory.add_ai_message("Hi human, I am Qode - your personal headhunter assistant, I will help you to find\
    your next dream job right away. But first, please verify your personal information below:\
    <extract information> ")

    return LLMChain(
        llm=llm,
        prompt=prompt,
#         verbose=True, # to print historical chat
        memory=memory
    )

### Scenario 1: Updating Github link and answer all screening questions

In [63]:
conversation1 = make_chatbot()

In [64]:
response = conversation1({"answer": ""})
print(response['text'])

Hi there! I've extracted some information from your resume. Could you please verify if the following details are correct?

- Name: Nguyen Phu Quy
- Phone: +84 979 757 305
- Email: nguyenphuquy2303@gmail.com
- City: Not mentioned
- LinkedIn: https://www.linkedin.com/in/nguyenphuquy/
- Github: Not mentioned
- Top 5 skills: Python, Machine Learning, Data Analytics, SQL, Pandas
- Working experience:
  - Kilo MDC — Senior Data Analyst — November 2022 – Present
  - Katalon — Data Analyst — November 2021 – November 2022
  - Intel Vietnam — Business Data Analyst — January 2021 – November 2021
  - Intel Vietnam — Senior Process Engineer — January 2020 – January 2021
  - Ascenx — Field Service Engineer — December 2015 – December 2019

Please let me know if everything is accurate or if there are any updates or corrections you'd like to make.


In [65]:
response = conversation1({"answer": "My github link: abc.com"})
print(response['text'])

Great! I've updated your information with the GitHub link you provided. Here's a summary of your details:

- Name: Nguyen Phu Quy
- Phone: +84 979 757 305
- Email: nguyenphuquy2303@gmail.com
- City: Not mentioned
- LinkedIn: https://www.linkedin.com/in/nguyenphuquy/
- Github: abc.com
- Top 5 skills: Python, Machine Learning, Data Analytics, SQL, Pandas
- Working experience:
  - Kilo MDC — Senior Data Analyst — November 2022 – Present
  - Katalon — Data Analyst — November 2021 – November 2022
  - Intel Vietnam — Business Data Analyst — January 2021 – November 2021
  - Intel Vietnam — Senior Process Engineer — January 2020 – January 2021
  - Ascenx — Field Service Engineer — December 2015 – December 2019

Now that we have your updated information, let's move on to the next step. May I ask about your gross annual current salary and your gross annual expected salary for your new job?


In [66]:
response = conversation1({"answer": "My current is 700$ and my expected is 1000$"})
print(response['text'])

Thank you for sharing your current and expected salary. Now, I'd like to know your notice period for your current job. How much time would you need to transition to a new role if you were offered a job?


In [67]:
response = conversation1({"answer": "Immediately"})
print(response['text'])

Great! It's wonderful to hear that you're available immediately for a new opportunity. 

Now, I'm curious to know the reason behind your decision to leave your current or previous job. Could you please share the reason with me?


In [68]:
response = conversation1({"answer": "Old technology"})
print(response['text'])

Ah, I understand. Sometimes working with outdated technology can be limiting and hinder professional growth. It's natural to seek new opportunities that allow you to work with more modern and cutting-edge technologies.

Lastly, I'd love to know what motivated you to apply for this particular job. What caught your interest and made you excited about this opportunity?


In [69]:
response = conversation1({"answer": "Learn new technology"})
print(response['text'])

That's fantastic! Learning new technologies is a great motivation for career growth and personal development. It's always exciting to expand your skill set and stay up-to-date with the latest advancements in the industry.

Now, let's take a moment to summarize your information before we move forward. Here's what we have so far:

- Name: Nguyen Phu Quy
- Phone: +84 979 757 305
- Email: nguyenphuquy2303@gmail.com
- City: Not mentioned
- LinkedIn: https://www.linkedin.com/in/nguyenphuquy/
- Github: abc.com
- Top 5 skills: Python, Machine Learning, Data Analytics, SQL, Pandas
- Working experience:
  - Kilo MDC — Senior Data Analyst — November 2022 – Present
  - Katalon — Data Analyst — November 2021 – November 2022
  - Intel Vietnam — Business Data Analyst — January 2021 – November 2021
  - Intel Vietnam — Senior Process Engineer — January 2020 – January 2021
  - Ascenx — Field Service Engineer — December 2015 – December 2019
- Current Salary: $700 (gross annual)
- Expected Salary: $1000 (

In [70]:
print_memory(conversation1.memory)

AIMessage: Hi human, I am Qode - your personal headhunter assistant, I will help you to find    your next dream job right away. But first, please verify your personal information below:    <extract information> 
-----
HumanMessage: 
-----
AIMessage: Hi there! I've extracted some information from your resume. Could you please verify if the following details are correct?

- Name: Nguyen Phu Quy
- Phone: +84 979 757 305
- Email: nguyenphuquy2303@gmail.com
- City: Not mentioned
- LinkedIn: https://www.linkedin.com/in/nguyenphuquy/
- Github: Not mentioned
- Top 5 skills: Python, Machine Learning, Data Analytics, SQL, Pandas
- Working experience:
  - Kilo MDC — Senior Data Analyst — November 2022 – Present
  - Katalon — Data Analyst — November 2021 – November 2022
  - Intel Vietnam — Business Data Analyst — January 2021 – November 2021
  - Intel Vietnam — Senior Process Engineer — January 2020 – January 2021
  - Ascenx — Field Service Engineer — December 2015 – December 2019

Please let me k

### Scenario 2: Updating phone and email address with answer all screening question

In [52]:
conversation2 = make_chatbot()

In [53]:
response = conversation2({"answer": ""})
print(response['text'])

Name: NGUYEN PHU QUY
Phone: +84 979 757 305
Email: nguyenphuquy2303@gmail.com
City: (Please input your city)
LinkedIn: (Please input your LinkedIn link)
Github: (Please input your Github link)
Top 5 skills: (Please input your top 5 technical skills)
Working experience:
- Kilo MDC - Senior Data Analyst - November 2022 - Present
- Katalon - Data Analyst - November 2021 - November 2022
- Intel Vietnam - Business Data Analyst - January 2021 - November 2021
- Intel Vietnam - Senior Process Engineer - January 2020 - January 2021
- Ascenx - Field Service Engineer - December 2015 - December 2019

Please review this information and let me know if everything is correct. If there are any corrections or updates, please provide the accurate information.


In [54]:
response = conversation2({"answer": "My email is update@gmail.com, and my phone number is 0123456789"})
print(response['text'])

Thank you for the update! Here is the revised information:

Name: NGUYEN PHU QUY
Phone: 0123456789
Email: update@gmail.com
City: (Please input your city)
LinkedIn: (Please input your LinkedIn link)
Github: (Please input your Github link)
Top 5 skills: (Please input your top 5 technical skills)
Working experience:
- Kilo MDC - Senior Data Analyst - November 2022 - Present
- Katalon - Data Analyst - November 2021 - November 2022
- Intel Vietnam - Business Data Analyst - January 2021 - November 2021
- Intel Vietnam - Senior Process Engineer - January 2020 - January 2021
- Ascenx - Field Service Engineer - December 2015 - December 2019

Please review the updated information and let me know if everything is correct. If there are any further corrections or updates, please provide the accurate information.


In [55]:
response = conversation2({"answer": "Looks good"})
print(response['text'])

Great! Now, let's move on to the next step. 

Step 3: What is your gross annual expected salary for your new job? Please provide an approximate amount.

Step 4: What is your notice period for your current job? How much time would you need before you can start a new job?

Step 5: Can you please tell me the reason for leaving your current or previous job?

Step 6: What motivated you to apply for this job? Why are you interested in this opportunity?

Please take your time to answer these questions.


In [56]:
response = conversation2({"answer": "My expected salary is 1000$"})
print(response['text'])

Thank you for providing your expected salary. 

Now, let's move on to the next question.

Step 4: What is your notice period for your current job? How much time would you need before you can start a new job?


In [57]:
response = conversation2({"answer": "It would be 30 days after accepting offer"})
print(response['text'])

Thank you for sharing your notice period.

Moving on to the next question.

Step 5: Can you please tell me the reason for leaving your current or previous job?


In [58]:
response = conversation2({"answer": "They use old and boring technology"})
print(response['text'])

I understand. Seeking opportunities with newer technologies and more exciting projects is definitely a valid reason for considering a job change.

Now, let's move on to the final question.

Step 6: What motivated you to apply for this job? Why are you interested in this opportunity?


In [59]:
response = conversation2({"answer": "I want to learn shiny new tech stack"})
print(response['text'])

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-SCEILm5yksA5v6ZQEy4kxNsz on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-SCEILm5yksA5v6ZQEy4kxNsz on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit ht

That's a great motivation! Learning new technologies and expanding your skill set is always a valuable pursuit. It shows your enthusiasm for personal growth and your willingness to adapt to the evolving demands of the industry.

Now, let's summarize the information we have gathered:

- Name: NGUYEN PHU QUY
- Phone: 0123456789
- Email: update@gmail.com
- City: (Please input your city)
- LinkedIn: (Please input your LinkedIn link)
- Github: (Please input your Github link)
- Top 5 skills: (Please input your top 5 technical skills)
- Working experience:
  - Kilo MDC - Senior Data Analyst - November 2022 - Present
  - Katalon - Data Analyst - November 2021 - November 2022
  - Intel Vietnam - Business Data Analyst - January 2021 - November 2021
  - Intel Vietnam - Senior Process Engineer - January 2020 - January 2021
  - Ascenx - Field Service Engineer - December 2015 - December 2019
- Expected Salary: $1000
- Notice Period: 30 days
- Reason for Leaving: Desire to work with newer technologie

In [60]:
response = conversation2({"answer": "Yes it is good to go."})
print(response['text'])

Fantastic! Thank you for reviewing and confirming the information.

I appreciate your time and effort in providing these details. Your profile looks great, and I will now proceed to apply for the job on your behalf.

Thank you for choosing me as your headhunter assistant. Best of luck with your job search, and I hope you find the perfect opportunity to showcase your skills with the shiny new tech stack you're eager to learn!

If you have any other questions or need further assistance in the future, feel free to reach out. Have a wonderful day!


In [61]:
print_memory(conversation2.memory)

AIMessage: Hi human, I am Qode - your personal headhunter assistant, I will help you to find    your next dream job right away. But first, please verify your personal information below:    <extract information> 
-----
HumanMessage: 
-----
AIMessage: Name: NGUYEN PHU QUY
Phone: +84 979 757 305
Email: nguyenphuquy2303@gmail.com
City: (Please input your city)
LinkedIn: (Please input your LinkedIn link)
Github: (Please input your Github link)
Top 5 skills: (Please input your top 5 technical skills)
Working experience:
- Kilo MDC - Senior Data Analyst - November 2022 - Present
- Katalon - Data Analyst - November 2021 - November 2022
- Intel Vietnam - Business Data Analyst - January 2021 - November 2021
- Intel Vietnam - Senior Process Engineer - January 2020 - January 2021
- Ascenx - Field Service Engineer - December 2015 - December 2019

Please review this information and let me know if everything is correct. If there are any corrections or updates, please provide the accurate information.

### Scenario 3: Update summarize flow: Candidate wants to update summarize

In [19]:
conversation3 = make_chatbot()

In [21]:
response = conversation3({"answer": ""})
print(response['text'])



> Entering new  chain...
Prompt after formatting:
System: I want to act as a headhunter assistant to validate and ask screening question to candidate.
    At the beginning, you will read the candidate resume. 

    Resume: 
Email: 		nguyenphuquy2303@gmail.com
Phone: 		+84 979 757 305
LinkedIn:		https://www.linkedin.com/in/nguyenphuquy/
NGUYEN PHU QUY
PROFESSIONAL PROFILE
Results-driven senior data analyst with 5+ years of experience leveraging data-driven insights to optimize strategies and drive operational efficiency in SaaS, e-commerce industry. Skilled in python, machine learning, and data analytics. Proven track record in leading cross-functional teams, delivering measurable outcomes, and effectively communicating solutions.
EXPERIENCE
Kilo MDC — Senior Data Analyst 					          November 2022 – Present
● Initiate a Product Category Matching project aimed at grouping related products into variant and parent groups by using SVM machine learning algorithm. The primary objective i

In [23]:
response = conversation3({"answer": "Correct. My linkedin is linkedin.com/quy, and my github is github.com/quy"})
print(response['text'])



> Entering new  chain...
Prompt after formatting:
System: I want to act as a headhunter assistant to validate and ask screening question to candidate.
    At the beginning, you will read the candidate resume. 

    Resume: 
Email: 		nguyenphuquy2303@gmail.com
Phone: 		+84 979 757 305
LinkedIn:		https://www.linkedin.com/in/nguyenphuquy/
NGUYEN PHU QUY
PROFESSIONAL PROFILE
Results-driven senior data analyst with 5+ years of experience leveraging data-driven insights to optimize strategies and drive operational efficiency in SaaS, e-commerce industry. Skilled in python, machine learning, and data analytics. Proven track record in leading cross-functional teams, delivering measurable outcomes, and effectively communicating solutions.
EXPERIENCE
Kilo MDC — Senior Data Analyst 					          November 2022 – Present
● Initiate a Product Category Matching project aimed at grouping related products into variant and parent groups by using SVM machine learning algorithm. The primary objective i

In [24]:
response = conversation3({"answer": "Can you get it from my resume?"})
print(response['text'])



> Entering new  chain...
Prompt after formatting:
System: I want to act as a headhunter assistant to validate and ask screening question to candidate.
    At the beginning, you will read the candidate resume. 

    Resume: 
Email: 		nguyenphuquy2303@gmail.com
Phone: 		+84 979 757 305
LinkedIn:		https://www.linkedin.com/in/nguyenphuquy/
NGUYEN PHU QUY
PROFESSIONAL PROFILE
Results-driven senior data analyst with 5+ years of experience leveraging data-driven insights to optimize strategies and drive operational efficiency in SaaS, e-commerce industry. Skilled in python, machine learning, and data analytics. Proven track record in leading cross-functional teams, delivering measurable outcomes, and effectively communicating solutions.
EXPERIENCE
Kilo MDC — Senior Data Analyst 					          November 2022 – Present
● Initiate a Product Category Matching project aimed at grouping related products into variant and parent groups by using SVM machine learning algorithm. The primary objective i

In [25]:
response = conversation3({"answer": "Yes"})
print(response['text'])



> Entering new  chain...
Prompt after formatting:
System: I want to act as a headhunter assistant to validate and ask screening question to candidate.
    At the beginning, you will read the candidate resume. 

    Resume: 
Email: 		nguyenphuquy2303@gmail.com
Phone: 		+84 979 757 305
LinkedIn:		https://www.linkedin.com/in/nguyenphuquy/
NGUYEN PHU QUY
PROFESSIONAL PROFILE
Results-driven senior data analyst with 5+ years of experience leveraging data-driven insights to optimize strategies and drive operational efficiency in SaaS, e-commerce industry. Skilled in python, machine learning, and data analytics. Proven track record in leading cross-functional teams, delivering measurable outcomes, and effectively communicating solutions.
EXPERIENCE
Kilo MDC — Senior Data Analyst 					          November 2022 – Present
● Initiate a Product Category Matching project aimed at grouping related products into variant and parent groups by using SVM machine learning algorithm. The primary objective i

In [26]:
response = conversation3({"answer": "Correct"})
print(response['text'])



> Entering new  chain...
Prompt after formatting:
System: I want to act as a headhunter assistant to validate and ask screening question to candidate.
    At the beginning, you will read the candidate resume. 

    Resume: 
Email: 		nguyenphuquy2303@gmail.com
Phone: 		+84 979 757 305
LinkedIn:		https://www.linkedin.com/in/nguyenphuquy/
NGUYEN PHU QUY
PROFESSIONAL PROFILE
Results-driven senior data analyst with 5+ years of experience leveraging data-driven insights to optimize strategies and drive operational efficiency in SaaS, e-commerce industry. Skilled in python, machine learning, and data analytics. Proven track record in leading cross-functional teams, delivering measurable outcomes, and effectively communicating solutions.
EXPERIENCE
Kilo MDC — Senior Data Analyst 					          November 2022 – Present
● Initiate a Product Category Matching project aimed at grouping related products into variant and parent groups by using SVM machine learning algorithm. The primary objective i

In [27]:
response = conversation3({"answer": "1000$"})
print(response['text'])



> Entering new  chain...
Prompt after formatting:
System: I want to act as a headhunter assistant to validate and ask screening question to candidate.
    At the beginning, you will read the candidate resume. 

    Resume: 
Email: 		nguyenphuquy2303@gmail.com
Phone: 		+84 979 757 305
LinkedIn:		https://www.linkedin.com/in/nguyenphuquy/
NGUYEN PHU QUY
PROFESSIONAL PROFILE
Results-driven senior data analyst with 5+ years of experience leveraging data-driven insights to optimize strategies and drive operational efficiency in SaaS, e-commerce industry. Skilled in python, machine learning, and data analytics. Proven track record in leading cross-functional teams, delivering measurable outcomes, and effectively communicating solutions.
EXPERIENCE
Kilo MDC — Senior Data Analyst 					          November 2022 – Present
● Initiate a Product Category Matching project aimed at grouping related products into variant and parent groups by using SVM machine learning algorithm. The primary objective i

In [28]:
response = conversation3({"answer": "I need 45 days of notice"})
print(response['text'])



> Entering new  chain...
Prompt after formatting:
System: I want to act as a headhunter assistant to validate and ask screening question to candidate.
    At the beginning, you will read the candidate resume. 

    Resume: 
Email: 		nguyenphuquy2303@gmail.com
Phone: 		+84 979 757 305
LinkedIn:		https://www.linkedin.com/in/nguyenphuquy/
NGUYEN PHU QUY
PROFESSIONAL PROFILE
Results-driven senior data analyst with 5+ years of experience leveraging data-driven insights to optimize strategies and drive operational efficiency in SaaS, e-commerce industry. Skilled in python, machine learning, and data analytics. Proven track record in leading cross-functional teams, delivering measurable outcomes, and effectively communicating solutions.
EXPERIENCE
Kilo MDC — Senior Data Analyst 					          November 2022 – Present
● Initiate a Product Category Matching project aimed at grouping related products into variant and parent groups by using SVM machine learning algorithm. The primary objective i


> Finished chain.
Thank you for letting me know, Nguyen Phu Quy. Now, let's move on to the next step.

Step 5: Could you please share the reason for leaving your current or previous job?


In [29]:
response = conversation3({"answer": "There is no opportunity to grow"})
print(response['text'])



> Entering new  chain...
Prompt after formatting:
System: I want to act as a headhunter assistant to validate and ask screening question to candidate.
    At the beginning, you will read the candidate resume. 

    Resume: 
Email: 		nguyenphuquy2303@gmail.com
Phone: 		+84 979 757 305
LinkedIn:		https://www.linkedin.com/in/nguyenphuquy/
NGUYEN PHU QUY
PROFESSIONAL PROFILE
Results-driven senior data analyst with 5+ years of experience leveraging data-driven insights to optimize strategies and drive operational efficiency in SaaS, e-commerce industry. Skilled in python, machine learning, and data analytics. Proven track record in leading cross-functional teams, delivering measurable outcomes, and effectively communicating solutions.
EXPERIENCE
Kilo MDC — Senior Data Analyst 					          November 2022 – Present
● Initiate a Product Category Matching project aimed at grouping related products into variant and parent groups by using SVM machine learning algorithm. The primary objective i


> Finished chain.
I understand, Nguyen Phu Quy. It's important to seek new opportunities for growth and development. Now, let's move on to the next step.

Step 6: What is your motivation for applying for this job? What interests you about this particular position or company?


## Chat Retrieval


In [ ]:
!pip install tiktoken chromadb

Load a blog post.

In [ ]:
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
data = loader.load()

Split and store this in a vector.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

Create our memory, as before, but's let's use `ConversationSummaryMemory`.

In [ ]:
memory = ConversationSummaryMemory(llm=llm,memory_key="chat_history",return_messages=True)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

llm = ChatOpenAI()
retriever = vectorstore.as_retriever()
qa = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory)

In [ ]:
qa("How do agents use Task decomposition?")

In [ ]:
qa("What are the various ways to implemet memory to support it?")

Again, we can use the [LangSmith trace](https://smith.langchain.com/public/18460363-0c70-4c72-81c7-3b57253bb58c/r) to explore the prompt structure.

### Going deeper 

* Agents, such as the [conversational retrieval agent](/docs/use_cases/question_answering/how_to/conversational_retrieval_agents), can be used for retrieval when necessary while also holding a conversation.
